# Training

In [1]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Trainer

In [2]:
# bsize = batch size
config = ColBERTConfig(
        bsize=64,
        root=r"../retrain_colbert",
        nbits=2,
    
    )

In [3]:
# nranks = number of gpu
with Run().context(RunConfig(nranks=1, experiment="msmarco")):
    trainer = Trainer(
        triples= r"../data/triples.train.small.id.json",
        queries= r"../data/queries.train.tsv",
        collection= r"../data/collection.tsv",
        config=config,
    )

    checkpoint_path = trainer.train()

    print(f"Saved checkpoint to {checkpoint_path}...")

#> Starting...
#> Joined...
Saved checkpoint to None...


In [ ]:
config

#  Trying to Load Torch Extension

In [6]:
from torch.utils.cpp_extension import load
import os
import pathlib

In [11]:
decompress_residuals_cpp = load(
    name="decompress_residuals_cpp",
    sources=[
        r'colbert/indexing/codecs/decompress_residuals.cpp',
        r"colbert/indexing/codecs/decompress_residuals.cu",
    ],
    verbose=False,
)

NameError: name 'cls' is not defined

In [4]:
packbits_cpp = load(
    name="packbits_cpp",
    sources=[
        r'D:\Documents\Python_Scripts\class\UCB\splade-colBERT\ColBERT\colbert\indexing\codecs\packbits.cpp',
        r'D:\Documents\Python_Scripts\class\UCB\splade-colBERT\ColBERT\colbert\indexing\codecs\packbits.cu',
    ],
    verbose=False,
)

ImportError: DLL load failed while importing packbits_cpp: The specified module could not be found.

# Indexing

In [1]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Indexer

In [20]:
# bsize = batch size
config = ColBERTConfig(
        bsize = 64,
        resume = False,
        checkpoint = "experiments/msmarco/colbert",
        collection = "../data/collection.tsv",
        index_name = "msmarco.nbits=2",
        root = "experiments",
        experiment = "msmarco",
        name = "",
        rank = 0,
        nranks = 1,
        amp = True,
        gpus = 1,
    )

In [25]:
with Run().context(RunConfig(nranks=1, experiment="msmarco")):
    indexer = Indexer(checkpoint=r"experiments/msmarco/colbert", config=config)
    indexer.index(name="msmarco.nbits=2", collection=r"../data/collection.tsv", overwrite='resume')



[Oct 08, 02:05:14] #> Note: Output directory /home/ubuntu/splade-colBERT/ColBERT/experiments/msmarco/indexes/msmarco.nbits=2 already exists


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "nbits": 1,
    "kmeans_niters": 4,
    "resume": true,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 500000,
    "save_every": null,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 220,
    "mask_punctuation": true,
    "checkpoint": "experiments\/msmarco\/colbert",
    "triples": "..\/data\/triples.train.small.id.json",
    "collection": "..\/data\/collection.tsv",
    "qu

16it [00:00, 155.47it/s]

[Oct 08, 02:06:21] [0] 		 #> Found chunk 0 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 1 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 2 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 3 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 4 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 5 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 6 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 7 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 8 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 9 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 10 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 11 in the in

48it [00:00, 155.25it/s]

[Oct 08, 02:06:21] [0] 		 #> Found chunk 32 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 33 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 34 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 35 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 36 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 37 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 38 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 39 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 40 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 41 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 42 in the index already, skipping encoding...
[Oct 08, 02:06:21] [0] 		 #> Found chunk 43

...

100%|██████████| 354/354 [00:00<00:00, 390.34it/s]

[Oct 08, 07:14:42] [0] 		 Sorting codes...
[Oct 08, 07:15:53] [0] 		 Getting unique codes...
[Oct 08, 07:15:54] #> Optimizing IVF to store map from centroids to list of pids..
[Oct 08, 07:15:54] #> Building the emb2pid mapping..
[Oct 08, 07:16:12] len(emb2pid) = 597760090



100%|██████████| 262144/262144 [01:41<00:00, 2581.49it/s]

[Oct 08, 07:18:02] #> Saved optimized IVF to /home/ubuntu/splade-colBERT/ColBERT/experiments/msmarco/indexes/msmarco.nbits=2/ivf.pid.pt
[Oct 08, 07:18:02] [0] 		 #> Saving the indexing metadata to /home/ubuntu/splade-colBERT/ColBERT/experiments/msmarco/indexes/msmarco.nbits=2/metadata.json ..


#> Joined...


# Retrieval

In [26]:
from colbert.data import Queries
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Searcher

In [28]:
config

ColBERTConfig(ncells=None, centroid_score_threshold=None, ndocs=None, index_path=None, nbits=1, kmeans_niters=4, resume=False, similarity='cosine', bsize=64, accumsteps=1, lr=3e-06, maxsteps=500000, save_every=None, warmup=None, warmup_bert=None, relu=False, nway=2, use_ib_negatives=False, reranker=False, distillation_alpha=1.0, ignore_scores=False, query_maxlen=32, attend_to_mask_tokens=False, interaction='colbert', dim=128, doc_maxlen=220, mask_punctuation=True, checkpoint='experiments/msmarco/colbert', triples=None, collection='../data/collection.tsv', queries=None, index_name='msmarco.nbits=2', overwrite=False, root='experiments', experiment='msmarco', index_root=None, name='', rank=0, nranks=1, amp=True, gpus=1)

In [27]:
with Run().context(RunConfig(nranks=1, experiment="msmarco")):
    searcher = Searcher(index="msmarco.nbits=2", config=config)
    queries = Queries("../data/queries.dev.tsv")
    ranking = searcher.search_all(queries, k=100)
    ranking.save("msmarco.nbits=2.ranking.tsv")

[Oct 08, 07:33:26] #> Loading collection...
0M 1M 2M 3M 4M 5M 6M 7M 8M 
[Oct 08, 07:33:53] #> Loading codec...
[Oct 08, 07:33:53] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Oct 08, 07:33:53] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Oct 08, 07:33:53] #> Loading IVF...
[Oct 08, 07:33:54] #> Loading doclens...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 354/354 [00:00<00:00, 810.38it/s]


[Oct 08, 07:33:55] #> Loading codes and residuals...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 354/354 [01:30<00:00,  3.93it/s]


[Oct 08, 07:35:26] #> Loading the queries from ../data/queries.dev.tsv ...
[Oct 08, 07:35:26] #> Got 101093 queries. All QIDs are unique.



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101093/101093 [38:39<00:00, 43.59it/s]




[Oct 08, 08:15:08] #> Creating directory /home/ubuntu/splade-colBERT/ColBERT/experiments/msmarco/none/2022-10/08/01.51.47 


[Oct 08, 08:15:25] #> Saved ranking of 101093 queries and 10109300 lines to /home/ubuntu/splade-colBERT/ColBERT/experiments/msmarco/none/2022-10/08/01.51.47/msmarco.nbits=2.ranking.tsv


# Evaluate

In [33]:
!python -m utility.evaluate.msmarco_passages \
--ranking "experiments/msmarco/none/2022-10/08/01.51.47/msmarco.nbits=2.ranking.tsv" \
--qrels "../data/qrels.train.tsv"

[Oct 08, 08:19:57] #> Loading QRELs from ../data/qrels.train.tsv ..
#> Reading ../data/qrels.train.tsv
 95%|█████████████▎| 9.5908842086792/10.09896469116211 [00:01<00:00,  7.66MiB/s]
[Oct 08, 08:19:58] #> Loading ranked lists from experiments/msmarco/none/2022-10/08/01.51.47/msmarco.nbits=2.ranking.tsv ..
#> Reading experiments/msmarco/none/2022-10/08/01.51.47/msmarco.nbits=2.ranking.tsv
100%|██████████| 261.58467292785645/261.58467292785645 [00:17<00:00, 15.02MiB/s]
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/colbert/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ubuntu/miniconda3/envs/colbert/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/splade-colBERT/ColBERT/utility/evaluate/msmarco_passages.py", line 126, in <module>
    main(args)
  File "/home/ubuntu/splade-colBERT/ColBERT/utility/evaluate/msmarco_passages.py", line 44, in main
  

In [38]:
!python -m utility.evaluate.msmarco_passages \
--ranking "experiments/msmarco/none/2022-10/08/01.51.47/msmarco.nbits=2.ranking.tsv" \
--qrels "../data/qrels.dev.tsv"

[Oct 08, 08:22:48] #> Loading QRELs from ../data/qrels.dev.tsv ..
#> Reading ../data/qrels.dev.tsv
 95%|██████████▍| 1.089432716369629/1.1459598541259766 [00:00<00:00,  7.07MiB/s]
[Oct 08, 08:22:48] #> Loading ranked lists from experiments/msmarco/none/2022-10/08/01.51.47/msmarco.nbits=2.ranking.tsv ..
#> Reading experiments/msmarco/none/2022-10/08/01.51.47/msmarco.nbits=2.ranking.tsv
100%|██████████| 261.58467292785645/261.58467292785645 [00:17<00:00, 14.94MiB/s]
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/colbert/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ubuntu/miniconda3/envs/colbert/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/splade-colBERT/ColBERT/utility/evaluate/msmarco_passages.py", line 126, in <module>
    main(args)
  File "/home/ubuntu/splade-colBERT/ColBERT/utility/evaluate/msmarco_passages.py", line 44, in main
    as

# Correcting Triple Input

In [2]:
from colbert.evaluation.loaders import *

In [3]:
from tqdm.auto import tqdm

In [15]:
param = {
    'triples': '../data/triples.train.small.tsv',
    'queries': '../data/queries.train.tsv',
    'collection': '../data/collection.tsv'
}

In [ ]:
def load_queries(queries_path):
    queries = OrderedDict()

    print_message("#> Loading the queries from", queries_path, "...")

    with open(queries_path, encoding='utf-8') as f:
        for line in f:
            qid, query, *_ = line.replace("\xa0", " ").strip().split('\t')
            qid = int(qid)

            assert (qid not in queries), ("Query QID", qid, "is repeated!")
            queries[re.sub('[^ 0-9a-zA-Z_-]', '', query.strip(" "))] = qid

    print_message("#> Got", len(queries), "queries. All QIDs are unique.\n")

    return queries

In [ ]:
def load_collection(collection_path):
    print_message("#> Loading collection...")

    collection = {}

    with open(collection_path, encoding="utf-8") as f:
        for line_idx, line in enumerate(f):
            if line_idx % (1000*1000) == 0:
                print(f'{line_idx // 1000 // 1000}M', end=' ', flush=True)

            pid, passage, *rest = line.strip('\n\r ').split('\t')
            assert pid == 'id' or int(pid) == line_idx

            if len(rest) >= 1:
                title = rest[0]
                passage = title + ' | ' + passage

            collection[passage] = line_idx

    print()

    return collection

In [ ]:
import re

In [ ]:
queries = load_queries(param['queries'])

[Sep 26, 18:41:38] #> Loading the queries from ../data/queries.train.tsv ...
[Sep 26, 18:41:39] #> Got 806349 queries. All QIDs are unique.



In [ ]:
collection = load_collection(param['collection'])

[Sep 26, 18:41:39] #> Loading collection...
0M 1M 2M 3M 4M 5M 6M 7M 8M 


In [ ]:
global err_text
err_text = ""
def get_id(text, data):
    err_text = text
    _id = data.get(exceptions.get(text, text), None)
    if _id is None: 
        text = text.replace("\xa0",' ')
        _id = data.get(exceptions.get(text, text), None)
    if _id is None: 
        n_text = text.strip(' ')
        _id = data.get(exceptions.get(n_text, n_text), None)
    if _id is None: 
        n_text = re.sub('[^ 0-9a-zA-Z_-]', '', n_text)
        _id = data.get(exceptions.get(n_text, n_text), None)
    if _id is None: 
        _id = data.get(text.strip(' '), None)
    
    if _id is None:
        print(text)
        raise Exception(text)
    return _id

In [ ]:
exceptions = {'divorce et sÃ©paration': 'divorce et séparation',
 'what is intelÂ® vpro technology': 'what is intel® vpro technology',
 'what is aÂ\xa0shock wave': 'what is a shock wave',
 'Germanyâ\x80\x99s perspective, the Treaty of Versailles was a fair settlement for its national interests': 'Germany’s perspective, the Treaty of Versailles was a fair settlement for its national interests',
 'yesÃ¼n temÃ¼r khan emperor taiding of yuan': 'yesün temür khan emperor taiding of yuan',
 ' The vitamin that prevents beriberi is ': ' The vitamin that prevents beriberi is',
 ' phosphates as food ingredients ': ' phosphates as food ingredients',
 ' who invented the periodic table ': ' who invented the periodic table',
 'what does bokmÃ¥l mean': 'what does bokmål mean',
 'which action should youÂ\xa0never take when selecting quotations': 'which action should you never take when selecting quotations',
 'dermatitis, anemia, convulsions, depressions, and confusion are all signs of a vitamin _________Â\xa0deficiency.': 'dermatitis anemia convulsions depressions and confusion are all signs of a vitamin _________ deficiency',
 ' In humans, the normal set point for body temperature is ': 'In humans the normal set point for body temperature is',
 'what did you notice about the relationship between pressure and volume when the temperatureÂ\xa0 is held constant?': 'what did you notice about the relationship between pressure and volume when the temperature  is held constant',
 'the Â\xa0____________Â\xa0 that vibrates like a drum when sound waves hit.': 'the  ____________  that vibrates like a drum when sound waves hit',
 'what is composition?Â\xa0 why is composition important?': 'what is composition  why is composition important',
 'the lithosphere consists of Â\xa0____________': 'the lithosphere consists of  ____________',
 "what is a 'cost engineer ": 'what is a cost engineer',
 'A simple way to save with a competitive interest rate. Your Personal Savings account earns interest daily and is posted to your account monthly. You can easily set up recurring transfers from your current bank accounts to your Personal Savings account.â\x80\xa0. Just deposit your savings and watch it grow. Your Personal Savings account earns interest daily and is posted to your account monthly. You can easily set up recurring transfers from your current bank accounts to your Personal Savings account.â\x80\xa0. ': 'A simple way to save with a competitive interest rate. Your Personal Savings account earns interest daily and is posted to your account monthly. You can easily set up recurring transfers from your current bank accounts to your Personal Savings account.â\x80\xa0. Just deposit your savings and watch it grow. Your Personal Savings account earns interest daily and is posted to your account monthly. You can easily set up recurring transfers from your current bank accounts to your Personal Savings account.â\x80\xa0.',
 "Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sira. Islamic jurisprudence is a complimentary expansion of the former by Islamic juris efinition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir ": "Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sira. Islamic jurisprudence is a complimentary expansion of the former by Islamic juris efinition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir",
 "Definition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir efinition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir ": "Definition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir efinition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir"}

In [ ]:
examples = []
with open(param['triples'], encoding='utf-8') as f:
    for line in tqdm(f):
        q_str, p_str_p, p_str_n = line.strip('\n').split('\t')
        qid = get_id(q_str, queries)
        pid_p = get_id(p_str_p, collection)
        pid_n = get_id(p_str_n, collection)
        example = [qid, pid_p, pid_n]
        examples.append(example)

0it [00:00, ?it/s]

In [ ]:
id_file = '../data/triples.train.small.id.json'
    
with open(id_file, 'w') as f:
    for example in examples:
        ujson.dump(example, f)
        f.write('\n')

output_path = f.name
print_message(f"#> Saved examples with {len(self.data)} lines to {f.name}")

NameError: name 'self' is not defined

In [ ]:
id_file = '../data/triples.train.small.id.tsv'
with open(id_file, 'r') as f:
    id_examples = ujson.load(f)

In [ ]:
len(id_examples)
len(id_examples[0])

In [ ]:
data = collection
# data = queries
qid = [k for k in data.keys() if "lamic Laws are made up of Shar" in k]
qid

["Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sira. Islamic jurisprudence is a complimentary expansion of the former by Islamic juris efinition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir",
 "Definition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø© Å\xa0arÄ«Ê¿ah) and Islamic jurisprudence (Ù\x81Ù\x82Ù\x87â\x80\x8e Fiqh). Shari'ah is seen as sacred and constitutes the Qur'an and Prophet Muhammad 's Sunnah (way), which is found in the Hadith and Sir efinition [edit]. Islamic Laws are made up of Shari'ah ('â\x80\x8eØ´Ø±Ù\x8aØ¹Ø

In [304]:
exceptions = {**exceptions, **{p_str_n: qid[1]}}
exceptions

{'divorce et sÃ©paration': 'divorce et séparation',
 'what is intelÂ® vpro technology': 'what is intel® vpro technology',
 'what is aÂ\xa0shock wave': 'what is a shock wave',
 'Germanyâ\x80\x99s perspective, the Treaty of Versailles was a fair settlement for its national interests': 'Germany’s perspective, the Treaty of Versailles was a fair settlement for its national interests',
 'yesÃ¼n temÃ¼r khan emperor taiding of yuan': 'yesün temür khan emperor taiding of yuan',
 ' The vitamin that prevents beriberi is ': ' The vitamin that prevents beriberi is',
 ' phosphates as food ingredients ': ' phosphates as food ingredients',
 ' who invented the periodic table ': ' who invented the periodic table',
 'what does bokmÃ¥l mean': 'what does bokmål mean',
 'which action should youÂ\xa0never take when selecting quotations': 'which action should you never take when selecting quotations',
 'dermatitis, anemia, convulsions, depressions, and confusion are all signs of a vitamin _________Â\xa0defic